In [4]:
from collections import deque

# 1. 맨해튼 거리 계산 함수
def calculate_manhattan_distance(a, b):
    """
    두 좌표 (a, b) 간의 맨해튼 거리를 계산하는 함수
    :param a: (x1, y1) 좌표 튜플
    :param b: (x2, y2) 좌표 튜플
    :return: 맨해튼 거리 (|x1 - x2| + |y1 - y2|)
    """
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


# 2. BFS를 이용한 최단 거리 맵 생성 함수
def compute_distances(start_x, start_y, N, obstacle_grid):
    """
    BFS를 사용하여 (start_x, start_y)에서 다른 모든 위치까지의 최단 거리를 계산하는 함수
    장애물(1)이 있는 경우 이동 불가

    :param start_x: 시작 위치의 x 좌표
    :param start_y: 시작 위치의 y 좌표
    :param N: 격자의 크기 (N x N)
    :param obstacle_grid: 장애물 정보를 담은 2D 리스트 (0: 이동 가능, 1: 장애물)
    :return: 최단 거리 맵 (2D 리스트)
    """
    INF = float('inf')  # 무한대를 나타내는 값
    distances = [[INF] * N for _ in range(N)]  # 모든 거리를 INF로 초기화
    distances[start_x][start_y] = 0  # 시작 위치의 거리는 0

    queue = deque([(start_x, start_y)])  # BFS 탐색을 위한 큐
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # 상하좌우 이동 방향 정의

    while queue:
        x, y = queue.popleft()

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < N and 0 <= ny < N and obstacle_grid[nx][ny] == 0:
                if distances[nx][ny] == INF:  # 아직 방문하지 않은 경우
                    distances[nx][ny] = distances[x][y] + 1
                    queue.append((nx, ny))

    return distances


# 3. 시야 계산 함수 (위쪽 방향)
def sight_up(x, y, N, is_test, warrior_count_grid, sight_map):
    """
    (x, y) 위치에서 위쪽 방향으로 시야를 확장하는 함수
    다이아몬드 모양으로 퍼져나가며 장애물을 만나면 시야가 차단됨

    :param x: 시작 x 좌표
    :param y: 시작 y 좌표
    :param N: 격자의 크기 (N x N)
    :param is_test: 테스트 여부 (True이면 시야 적용 후 원상복구)
    :param warrior_count_grid: 전사 배치 정보 (전사가 있는 칸은 1, 없으면 0)
    :param sight_map: 현재까지의 시야 정보
    """
    original_sight_map = [row[:] for row in sight_map]  # 기존 시야 맵 백업

    for dx in range(1, N):  # 위쪽 방향으로 확장 (dx: 거리)
        if x - dx < 0:
            break  # 격자 범위를 벗어나면 종료

        for dy in range(-dx, dx + 1):  # 다이아몬드 형태의 확장 범위
            ny = y + dy
            if 0 <= ny < N:  # y 범위 체크
                sight_map[x - dx][ny] += 1

                # 장애물 만나면 그 아래쪽은 더 이상 탐색 X
                if warrior_count_grid[x - dx][ny] == 1:
                    break

    if is_test:
        sight_map[:] = original_sight_map  # 원상복구


# 4. 시야 계산 함수 (아래쪽 방향)
def sight_down(x, y, N, is_test, warrior_count_grid, sight_map):
    """
    (x, y) 위치에서 아래쪽 방향으로 시야를 확장하는 함수
    위쪽 방향과 같은 논리로 동작
    """
    original_sight_map = [row[:] for row in sight_map]

    for dx in range(1, N):
        if x + dx >= N:
            break

        for dy in range(-dx, dx + 1):
            ny = y + dy
            if 0 <= ny < N:
                sight_map[x + dx][ny] += 1

                if warrior_count_grid[x + dx][ny] == 1:
                    break

    if is_test:
        sight_map[:] = original_sight_map


# 5. 시야 계산 함수 (왼쪽 방향)
def sight_left(x, y, N, is_test, warrior_count_grid, sight_map):
    """
    (x, y) 위치에서 왼쪽 방향으로 시야를 확장하는 함수
    위쪽 방향과 같은 논리로 동작
    """
    original_sight_map = [row[:] for row in sight_map]

    for dy in range(1, N):
        if y - dy < 0:
            break

        for dx in range(-dy, dy + 1):
            nx = x + dx
            if 0 <= nx < N:
                sight_map[nx][y - dy] += 1

                if warrior_count_grid[nx][y - dy] == 1:
                    break

    if is_test:
        sight_map[:] = original_sight_map


# 6. 시야 계산 함수 (오른쪽 방향)
def sight_right(x, y, N, is_test, warrior_count_grid, sight_map):
    """
    (x, y) 위치에서 오른쪽 방향으로 시야를 확장하는 함수
    위쪽 방향과 같은 논리로 동작
    """
    original_sight_map = [row[:] for row in sight_map]

    for dy in range(1, N):
        if y + dy >= N:
            break

        for dx in range(-dy, dy + 1):
            nx = x + dx
            if 0 <= nx < N:
                sight_map[nx][y + dy] += 1

                if warrior_count_grid[nx][y + dy] == 1:
                    break

    if is_test:
        sight_map[:] = original_sight_map

print('*'*1000)
from collections import deque

# 시야를 확장하는 방향 함수
def expand_sight(x, y, N, warrior_count_grid, sight_map):
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # 상, 하, 좌, 우
    # 시야를 확장하는 다이아몬드 형태로 진행
    for dx in range(1, N):
        if x - dx < 0 or x + dx >= N:  # 격자 범위를 벗어나면 종료
            break
        for dy in range(-dx, dx + 1):  # 다이아몬드 형태로 확장
            ny = y + dy
            if 0 <= ny < N:  # y 범위 체크
                sight_map[x - dx][ny] += 1  # 위쪽 방향으로 시야 확장
                sight_map[x + dx][ny] += 1  # 아래쪽 방향으로 시야 확장
                if warrior_count_grid[x - dx][ny] == 1 or warrior_count_grid[x + dx][ny] == 1:
                    return  # 전사를 만나면 시야 확장을 멈춤

# 시야를 계산하는 함수
def calculate_sight(N, warrior_count_grid):
    sight_map = [[0] * N for _ in range(N)]  # 시야 맵 초기화
    for x in range(N):
        for y in range(N):
            if warrior_count_grid[x][y] == 1:  # 전사가 있는 위치라면
                expand_sight(x, y, N, warrior_count_grid, sight_map)
    return sight_map

# 입력 데이터
N = 4  # 격자 크기 (4x4)
warrior_count_grid = [
    [1, 3, 3, 3],
    [3, 1, 0, 3],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
]

sight_map = calculate_sight(N, warrior_count_grid)

# 시야 맵 출력
for row in sight_map:
    print(' '.join(map(str, row)))




#4시간 27분
print('%'*100)
from collections import deque

# 1. 맨해튼 거리 계산 함수
def calculate_manhattan_distance(a, b):
    """
    두 좌표 (a, b) 간의 맨해튼 거리를 계산하는 함수
    :param a: (x1, y1) 좌표 튜플
    :param b: (x2, y2) 좌표 튜플
    :return: 맨해튼 거리 (|x1 - x2| + |y1 - y2|)
    """
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


# 2. BFS를 이용한 최단 거리 맵 생성 함수
def compute_distances(start_x, start_y, N, obstacle_grid):
    """
    BFS를 사용하여 (start_x, start_y)에서 다른 모든 위치까지의 최단 거리를 계산하는 함수
    장애물(1)이 있는 경우 이동할 수 없음

    :param start_x: 시작 위치의 x 좌표
    :param start_y: 시작 위치의 y 좌표
    :param N: 격자의 크기 (N x N)
    :param obstacle_grid: 장애물 정보를 담은 2D 리스트 (0: 이동 가능, 1: 장애물)
    :return: 최단 거리 맵 (2D 리스트)
    """
    INF = float('inf')  # 무한대를 나타내는 값
    distances = [[INF] * N for _ in range(N)]  # 모든 거리를 INF로 초기화
    distances[start_x][start_y] = 0  # 시작 위치의 거리는 0

    queue = deque([(start_x, start_y)])  # BFS 탐색을 위한 큐
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # 상하좌우 이동 방향 정의

    while queue:
        x, y = queue.popleft()

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < N and 0 <= ny < N and obstacle_grid[nx][ny] == 0:
                if distances[nx][ny] == INF:  # 아직 방문하지 않은 경우
                    distances[nx][ny] = distances[x][y] + 1
                    queue.append((nx, ny))

    return distances


# 3. 시야 확장 함수 (다이아몬드 형태)
# 시야 확장 함수 (다이아몬드 형태로 시야 확장)
def expand_sight(x, y, N, warrior_count_grid, sight_map):
    # 각 방향으로 다이아몬드 모양의 시야 확장
    for dx in range(1, N):  # 위 아래로 확장
        if x - dx < 0 or x + dx >= N:  # 격자 범위를 벗어나면 종료
            break
        for dy in range(-dx, dx + 1):  # 다이아몬드 형태로 확장
            ny = y + dy
            if 0 <= ny < N:  # y 범위 체크
                # 위쪽과 아래쪽 방향으로 시야 확장
                if warrior_count_grid[x - dx][ny] != 1:
                    sight_map[x - dx][ny] += 1
                if warrior_count_grid[x + dx][ny] != 1:
                    sight_map[x + dx][ny] += 1
                # 장애물 만나면 시야 확장 중지
                if warrior_count_grid[x - dx][ny] == 1 or warrior_count_grid[x + dx][ny] == 1:
                    return


# 시야 계산 함수
def calculate_sight(N, warrior_count_grid):
    sight_map = [[0] * N for _ in range(N)]  # 시야 맵 초기화
    for x in range(N):
        for y in range(N):
            if warrior_count_grid[x][y] == 1:  # 전사가 있는 위치라면
                expand_sight(x, y, N, warrior_count_grid, sight_map)
    return sight_map


# 입력 데이터
N = 4  # 격자 크기 (4x4)
warrior_count_grid = [
    [1, 3, 3, 3],
    [3, 1, 0, 3],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
]

# 시야 계산
sight_map = calculate_sight(N, warrior_count_grid)

# 시야 맵 출력
for row in sight_map:
    print(' '.join(map(str, row)))
#4시간 13분

****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************